# **Import Libraries**

In [1]:
import pyspark 
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import *
import os

# **Create Dataframe**

In [2]:
# You need to have Java installed to run Spark.
os.environ["JAVA_HOME"] = "C:\\Program Files\\Java\\jdk-22"


spark = SparkSession.builder.appName("Google Play Store Data").getOrCreate()

In [46]:
df = spark.read.csv("googleplaystore.csv", header=True, inferSchema=True)

In [38]:
# coiunt number of rows
df.count()

10841

In [39]:
# show first 3 rows
df.show(3)

+--------------------+--------------+------+-------+----+----------+----+-----+--------------+--------------------+----------------+-----------+------------+
|                 App|      Category|Rating|Reviews|Size|  Installs|Type|Price|Content Rating|              Genres|    Last Updated|Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+----------+----+-----+--------------+--------------------+----------------+-----------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M|   10,000+|Free|    0|      Everyone|        Art & Design| January 7, 2018|      1.0.0|4.0.3 and up|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967| 14M|  500,000+|Free|    0|      Everyone|Art & Design;Pret...|January 15, 2018|      2.0.0|4.0.3 and up|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510|8.7M|5,000,000+|Free|    0|      Everyone|        Art & Design|  August 1, 2018|      1.2.4|4.0.3 and up|
+--------------------+--------------+------+-------+

In [40]:
# Check schema
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



# **Data Cleaning step**

In [47]:
#dropping not important columns 

df = df.drop("size","Content Rating","Last Updated","Android Ver","Current Ver")
df.show(2)

+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|              Genres|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 10,000+|Free|    0|        Art & Design|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|500,000+|Free|    0|Art & Design;Pret...|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
only showing top 2 rows



In [48]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Genres: string (nullable = true)



In [49]:
unique_values_type = df.select('price').distinct()
unique_values_type.show()

+-------+
|  price|
+-------+
|  $1.29|
| $19.99|
|  $0.99|
|  $4.59|
|$400.00|
| $89.99|
| $24.99|
|  $1.96|
| $39.99|
| $12.99|
|  $8.99|
| $11.99|
| $14.99|
| $25.99|
|  $3.04|
| $15.46|
|  $2.50|
|      0|
|   5.0M|
|  $2.49|
+-------+
only showing top 20 rows



In [50]:
# Transform the DataFrame
df = df.withColumn("Rating", col("Rating").cast("float")) \
       .withColumn("Reviews", col("Reviews").cast("int")) \
       .withColumn("installs", regexp_replace("installs", "[^0-9]", "")) \
       .withColumn("installs", col("installs").cast("int")) \
       .withColumn("price", regexp_replace("price", r"\$", "")) \
       .withColumn("price", col("price").cast("float"))

In [51]:
df.show(3)

+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|                 App|      Category|Rating|Reviews|installs|Type|price|              Genres|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|   10000|Free|  0.0|        Art & Design|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|  500000|Free|  0.0|Art & Design;Pret...|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510| 5000000|Free|  0.0|        Art & Design|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
only showing top 3 rows



In [52]:
df.createOrReplaceTempView("apps")

In [56]:
results_sql = spark.sql("select * from apps limit 10")
results_sql.show()

+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|                 App|      Category|Rating|Reviews|installs|Type|price|              Genres|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|   10000|Free|  0.0|        Art & Design|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|  500000|Free|  0.0|Art & Design;Pret...|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510| 5000000|Free|  0.0|        Art & Design|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5| 215644|50000000|Free|  0.0|        Art & Design|
|Pixel Draw - Numb...|ART_AND_DESIGN|   4.3|    967|  100000|Free|  0.0|Art & Design;Crea...|
|Paper flowers ins...|ART_AND_DESIGN|   4.4|    167|   50000|Free|  0.0|        Art & Design|
|Smoke Effect Phot...|ART_AND_DESIGN|   3.8|    178|   50000|Free|  0.0|        Art & Design|
|    Infinite Painter|ART_AND_DESIGN|   4.1|  36815| 1000000

# **Top reviews give to the apps**

In [69]:
results_sql = spark.sql("""select App, SUM(Reviews) as TotalReviews
                         from apps
                         group by App
                         order by TotalReviews DESC limit 10
                        """)
results_sql.show()

+--------------------+------------+
|                 App|TotalReviews|
+--------------------+------------+
|           Instagram|   266241989|
|  WhatsApp Messenger|   207348304|
|      Clash of Clans|   179558781|
|Messenger – Text ...|   169932272|
|      Subway Surfers|   166331958|
|    Candy Crush Saga|   156993136|
|            Facebook|   156286514|
|         8 Ball Pool|    99386198|
|        Clash Royale|    92530298|
|            Snapchat|    68045010|
+--------------------+------------+

